# Agentic RAG 

## Dependencies

In [25]:
%pip install -q llama-index
%pip install -q python-dotenv
%pip install -q llama-index-embeddings-gemini
%pip install -q llama-index-llms-gemini
%pip install -q ipywidgets

In [26]:


from llama_index.core.agent import ReActAgent
from llama_index.llms.gemini import Gemini
from llama_index.core.tools import FunctionTool

In [27]:
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

## Creating basic Tool

In [28]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Setting the model

In [29]:
llm = Gemini(model='models/gemini-1.5-flash',temperature=0,api_key=GOOGLE_API_KEY)

## Creating the Agent

In [30]:
agent = ReActAgent.from_tools([multiply_tool,add_tool],llm=llm,verbose=True)

## Asking a simple question

In [31]:
response = agent.chat("What is 20+(2*4)? Use a tool to calculate every step.")

> Running step 314ee592-a0e6-4874-aee6-43cfb92a76a3. Step input: What is 20+(2*4)? Use a tool to calculate every step.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step e3d93085-4238-4b41-9408-bcd02cee37cb. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 71816f7b-d812-4f25-9fd5-28f32547048d. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 20 + (2 * 4) = 20 + 8 = 28


## New dependencies

In [32]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings

## Adding LLM Globally

In [33]:
Settings.llm = llm

## Setting the embedding model

In [34]:
from llama_index.embeddings.gemini import GeminiEmbedding

model_name = "models/embedding-001"

Settings.embed_model = em = GeminiEmbedding(
    model_name=model_name,
    api_key=GOOGLE_API_KEY
)

# Creating the vector store and retriving embeddings 

In [35]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [36]:
response = query_engine.query(
    "What was the total amount of the 2023 Canadian federal budget?"
)
print(response)

The total projected expenditures for the 2023 Canadian federal budget were $496.9 billion. 



In [37]:
from llama_index.core.tools import QueryEngineTool
budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget.",
)